In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm, trange
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertConfig

from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

torch.__version__

import transformers
from transformers import BertForTokenClassification, AdamW

from transformers import get_linear_schedule_with_warmup
transformers.__version__


from seqeval.metrics import f1_score, accuracy_score

In [2]:
class SentenceGetter(object):

    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["word"].values.tolist(),
                                                           s["pos"].values.tolist(),
                                                           s["NER"].values.tolist())]
        self.grouped = self.data.groupby("sentence").apply(agg_func)
        self.sentences = [s for s in self.grouped]

    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [3]:
path_ = r"C:\Users\LAB\jupyter\DeepLearning_project\data\NER\{}.xlsx"

buds_live = pd.read_excel(path_.format("buds_live"), encoding="UTF-8").fillna(method="ffill")
buds_live.word=buds_live.word.astype(str)

buds_live.head()

,Unnamed: 0,word,rule_label,IOB,sentence,enter,review,pos,NER
0,0,2020,NaN,O,0,0,0,Number,O
1,1,.,NaN,O,0,0,0,Punctuation,O
2,2,08,NaN,O,0,0,0,Number,O
3,3,.,NaN,O,0,0,0,Punctuation,O
4,4,07,NaN,O,0,0,0,Number,O


In [4]:
getter = SentenceGetter(buds_live)
getter.sentences[0]

[('2020', 'Number', 'O'),
 ('.', 'Punctuation', 'O'),
 ('08', 'Number', 'O'),
 ('.', 'Punctuation', 'O'),
 ('07', 'Number', 'O'),
 ('로켓', 'Noun', 'O'),
 ('와우', 'Noun', 'O'),
 ('아침', 'Noun', 'O'),
 ('6시', 'Number', 'O'),
 ('에', 'Foreign', 'O'),
 ('수령', 'Noun', 'O'),
 ('받다', 'Verb', 'O'),
 ('뒤', 'Noun', 'O'),
 ('페어', 'Noun', 'O'),
 ('링후', 'Noun', 'O'),
 ('약', 'Noun', 'O'),
 ('15분', 'Number', 'O'),
 ('간', 'Foreign', 'O'),
 ('사용', 'Noun', 'O'),
 ('후', 'Noun', 'O'),
 ('후기', 'Noun', 'O'),
 ('올리다', 'Verb', 'O'),
 ('.', 'Punctuation', 'O')]

In [5]:
sentences = [[word[0] for word in sentence] for sentence in getter.sentences]
sentences[0]

['2020',
 '.',
 '08',
 '.',
 '07',
 '로켓',
 '와우',
 '아침',
 '6시',
 '에',
 '수령',
 '받다',
 '뒤',
 '페어',
 '링후',
 '약',
 '15분',
 '간',
 '사용',
 '후',
 '후기',
 '올리다',
 '.']

In [6]:
labels = [[s[2] for s in sentence] for sentence in getter.sentences]
print(labels[0])

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [9]:
tag_values = list(set(buds_live["NER"].values))
tag_values.append("PAD")
tag2idx = {t: i for i, t in enumerate(tag_values)}

In [10]:
MAX_LEN = 70
bs = 32

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

In [12]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased', do_lower_case=False)

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [13]:
def tokenize_and_preserve_labels(sentence, text_labels):
    tokenized_sentence = []
    labels = []

    for word, label in zip(sentence, text_labels):

        # Tokenize the word and count # of subwords the word is broken into
        tokenized_word = tokenizer.tokenize(word)
        n_subwords = len(tokenized_word)

        # Add the tokenized word to the final tokenized word list
        tokenized_sentence.extend(tokenized_word)

        # Add the same label to the new list of labels `n_subwords` times
        labels.extend([label] * n_subwords)

    return tokenized_sentence, labels

In [14]:
tokenized_texts_and_labels = [
    tokenize_and_preserve_labels(sent, labs)
    for sent, labs in zip(sentences, labels)
]

In [15]:
tokenized_texts = [token_label_pair[0] for token_label_pair in tokenized_texts_and_labels]
labels = [token_label_pair[1] for token_label_pair in tokenized_texts_and_labels]

In [16]:
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=MAX_LEN, dtype="long", value=0.0,
                          truncating="post", padding="post")

In [17]:
for labels_ in labels:
    for label_ in labels_:
        if type(label_) == "NoneType":
            print(label_)

In [18]:
tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels],
                     maxlen=MAX_LEN, value=tag2idx["PAD"], padding="post",
                     dtype="long", truncating="post")

In [19]:
attention_masks = [[float(i != 0.0) for i in ii] for ii in input_ids]

In [20]:
tr_inputs, val_inputs, tr_tags, val_tags = train_test_split(input_ids, tags,
                                                            random_state=2018, test_size=0.1)
tr_masks, val_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=2018, test_size=0.1)

In [21]:
tr_inputs = torch.tensor(tr_inputs).to(torch.int64)
val_inputs = torch.tensor(val_inputs).to(torch.int64)
tr_tags = torch.tensor(tr_tags).to(torch.int64)
val_tags = torch.tensor(val_tags).to(torch.int64)
tr_masks = torch.tensor(tr_masks).to(torch.int64)
val_masks = torch.tensor(val_masks).to(torch.int64)

In [22]:
train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=bs)

valid_data = TensorDataset(val_inputs, val_masks, val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=bs)

In [23]:
model = BertForTokenClassification.from_pretrained(
    "bert-base-cased",
    num_labels=len(tag2idx),
    output_attentions = False,
    output_hidden_states = False
)

Downloading:   0%|          | 0.00/433 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cas

In [24]:
FULL_FINETUNING = True
if FULL_FINETUNING:
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    param_optimizer = list(model.classifier.named_parameters())
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]

optimizer = AdamW(
    optimizer_grouped_parameters,
    lr=3e-5,
    eps=1e-8
)

In [28]:
epochs = 30
max_grad_norm = 1.0

# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)


In [29]:
## Store the average loss after each epoch so we can plot them.
loss_values, validation_loss_values = [], []

for _ in trange(epochs, desc="Epoch"):
    # ========================================
    #               Training
    # ========================================
    # Perform one full pass over the training set.


    model.train()
    # 각 epoch마다 Reset.
    total_loss = 0

    # Training loop
    for step, batch in enumerate(train_dataloader):
    
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        model.zero_grad()
        # forward pass
        outputs = model(b_input_ids, token_type_ids=None,
                        attention_mask=b_input_mask, labels=b_labels)
        # get the loss
        loss = outputs[0]
        # Perform a backward pass to calculate the gradients.
        loss.backward()
        total_loss += loss.item()
        # Clip the norm of the gradient
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        # parameter 업데이트
        optimizer.step()
        # learning rate 업데이트
        scheduler.step()

    # 평균 loss 계산
    avg_train_loss = total_loss / len(train_dataloader)
    print("Average train loss: {}".format(avg_train_loss))

    # loss value 저장
    loss_values.append(avg_train_loss)


    # ========================================
    #               Validation
    # ========================================

    # evaluation mode로 전환
    model.eval()
    # Reset epoch.
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    predictions , true_labels = [], []
    for batch in valid_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch

        # 계산할 모델이 있거나, Gredient 의 상황 알리기,
        with torch.no_grad():
            # Forward pass에는 logit predictions.
            outputs = model(b_input_ids, token_type_ids=None,
                            attention_mask=b_input_mask, labels=b_labels)
        #  CPU로 이동
        logits = outputs[1].detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # test sentences accuracy .
        eval_loss += outputs[0].mean().item()
        predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
        true_labels.extend(label_ids)

    eval_loss = eval_loss / len(valid_dataloader)
    validation_loss_values.append(eval_loss)
    print("Validation loss: {}".format(eval_loss))
    pred_tags = [tag_values[p_i] for p, l in zip(predictions, true_labels)
                                 for p_i, l_i in zip(p, l) if tag_values[l_i] != "PAD"]
    valid_tags = [tag_values[l_i] for l in true_labels
                                  for l_i in l if tag_values[l_i] != "PAD"]
    print("Validation Accuracy: {}".format(accuracy_score(pred_tags, valid_tags)))
    print("Validation F1-Score: {}".format(f1_score([pred_tags], [valid_tags])))
    print()

Epoch:   0%|                                                                                    | 0/30 [00:00<?, ?it/s]

Average train loss: 0.6135273851266428


Epoch:   3%|██▎                                                                    | 1/30 [25:09<12:09:21, 1509.00s/it]

Validation loss: 0.6627909583704812
Validation Accuracy: 0.8733962720890825
Validation F1-Score: 0.02185792349726776

Average train loss: 0.6090371934305719


Epoch:   7%|████▋                                                                  | 2/30 [46:51<10:47:31, 1387.55s/it]

...

Validation loss: 0.7396984483514514
Validation Accuracy: 0.8629871701767127
Validation F1-Score: 0.04716981132075472

Average train loss: 0.45522958516072826


Epoch:  97%|████████████████████████████████████████████████████████████████████▋  | 29/30 [8:31:10<17:02, 1022.10s/it]

Validation loss: 0.7447442476238523
Validation Accuracy: 0.8617768094892277
Validation F1-Score: 0.046403712296983764

Average train loss: 0.4534198239821346


Epoch: 100%|███████████████████████████████████████████████████████████████████████| 30/30 [8:49:29<00:00, 1058.98s/it]

Validation loss: 0.7429871473993573
Validation Accuracy: 0.8637133865892036
Validation F1-Score: 0.047393364928909956

